In [8]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [9]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [10]:
df = pd.read_csv("../data/soybean.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'soybean'

In [11]:
X.shape

(675, 35)

In [12]:
pd.value_counts(y)

5     92
12    91
1     91
15    88
6     44
2     44
10    20
17    20
14    20
4     20
7     20
3     20
11    20
16    20
18    20
0     16
9     15
8     14
Name: count, dtype: int64

In [13]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [27]:
def fit_trees():
    learning_rate=1
    max_depth=6
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 3
    verbose = 1
    tolerance= 0.1

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 5
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

with open('./benchmark/'+dataset+'.csv','w') as f:
        f.writelines("")

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)

    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),gtgp.train_p)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),gtgp.test_p)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

25 97 1
51 207 2
73 293 3
90 364 4
108 424 5
129 501 6
149 583 7
166 646 8
186 738 9
207 823 10
222 882 11
237 941 12
255 1013 13
275 1091 14
295 1167 15
315 1245 16
322 1274 17
335 1325 18
352 1404 19
369 1481 20
382 1526 21
398 1582 22
412 1638 23
425 1697 24
433 1727 25
448 1784 26
465 1843 27
480 1914 28
493 1971 29
504 2016 30
516 2062 31
533 2127 32
550 2206 33
568 2280 34
581 2339 35
592 2380 36
609 2451 37
624 2518 38
633 2561 39
646 2616 40
657 2663 41
668 2714 42
680 2766 43
690 2806 44
700 2844 45
717 2907 46
728 2962 47
742 3024 48
748 3046 49
763 3115 50
771 3155 51
784 3202 52
794 3244 53
805 3307 54
814 3336 55
825 3383 56
836 3434 57
850 3496 58
861 3545 59
871 3593 60
881 3621 61
892 3674 62
900 3702 63
910 3746 64
919 3789 65
926 3822 66
936 3868 67
947 3921 68
957 3961 69
965 3995 70
970 4014 71
980 4052 72
990 4102 73
1003 4159 74
1010 4188 75
1017 4219 76
1031 4287 77
1039 4315 78
1051 4375 79
1060 4414 80
1074 4476 81
1083 4515 82
1090 4542 83
1104 4610 84
1113 46

# Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
with open('./benchmark_DC/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        clf = DecisionTreeClassifier()
        clf.fit(X_train,y_train)
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

        num_trees = 1
        depth = clf.tree_.max_depth
        num_nodes = clf.tree_.node_count

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        # train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        # test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [29]:
import xgboost

In [40]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=3)
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')
        # train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        # test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

In [42]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier(n_estimators=100)
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')
        # train_f1 = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        # test_f1 = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))


        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # rfc = RandomForestClassifier(n_estimators=100)
        rfc = RandomForestClassifier()
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')
        
        # train_f1 = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
        # test_f1 = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)